# Ejercicio: Fragmentador de capítulos en un documento

Define extensiones personalizadas para los objetos `Span` y `Doc`:

- Cada `Span` (capítulo) debe tener:
  - Su número (`numero`)
  - Su título (`titulo`)
  - Su longitud en tokens (`longitud`)

- El objeto `Doc` debe tener:
  - Una lista de todos los capítulos (`capitulos`)



In [12]:
import re
with open("libro1.txt", "r", encoding="utf-8") as archivo:
    lineas = archivo.readlines()[37:]  # Desde la línea 38
    texto = "".join(lineas)
import spacy
nlp = spacy.load("es_core_news_md")
doc = nlp(texto)
#importa la clase Span
from spacy.tokens import Span, Doc
#hagamos un span para cada capítulo. debe tener su número, su título y su longitud
if not Span.has_extension("numero"):
    # Registrar atributos personalizados en Span
    Span.set_extension("numero", default=None)
    '''Span.set_extension("titulo", default=None)
    Span.set_extension("longitud", default=None)
# Registrar atributos personalizados en Span
Span.set_extension("numero", default=None)'''
if not Span.has_extension("titulo"):
    # Registrar atributos personalizados en Span
    Span.set_extension("titulo", default=None)
if not Span.has_extension("longitud"):
    # Registrar atributos personalizados en Span
    Span.set_extension("longitud", default=None)
#Buscar encabezados de capítulo con regex: número al principio de línea seguido de texto
# Ejemplo: "1 El niño que vivió"
capitulos = list(re.finditer(r"(?m)^(\d+)\s+([A-Z][^\n]*)", texto))

# Lista para guardar la info de cada capítulo
capitulos_info = []

# Iterar por capítulos para construir spans
for i, match in enumerate(capitulos):
    numero = int(match.group(1))
    titulo = match.group(2).strip()

    # Índices del capítulo actual en caracteres
    start_char = match.end()

    # Determinar el final del capítulo: inicio del siguiente o final del texto
    if i + 1 < len(capitulos):
        end_char = capitulos[i + 1].start()
    else:
        end_char = len(texto)

    # Crear el span usando char_span
    span = doc.char_span(start_char, end_char, alignment_mode="expand")

    if span:
        # Atributos personalizados
        span._.numero = numero
        span._.titulo = titulo
        span._.longitud = len(span)

        capitulos_info.append(span)
    
resumen = "\n".join([
    f"{span._.numero} {span._.titulo} {span._.longitud}"
    for span in capitulos_info
])
resumen


# Crear un nuevo Doc con ese texto
doc_resumen = nlp.make_doc(resumen)

Crea un componente personalizado para añadir luego al pipeline, llamado `fragmentador_capitulos` que:

- Reciba un `Doc`
- Busque los capítulos utilizando una expresión regular, donde cada capítulo esté definido por un número seguido de su título.
- Cree un `Span` para cada capítulo con los atributos personalizados definidos.
- Guarde la lista de capítulos en el atributo `doc._.capitulos`.

In [25]:
from spacy.language import Language
@Language.component("fragmentador_capitulos")
def fragmentador_capitulos(doc):
    texto = doc.text

    # Buscar encabezados de capítulos: número + título
    capitulos = list(re.finditer(r"(?m)^(\d+)\s+([A-Z][^\n]*)", texto))
    spans = []

    for i, match in enumerate(capitulos):
        numero = int(match.group(1))
        titulo = match.group(2).strip()
        start_char = match.end()
        end_char = capitulos[i + 1].start() if i + 1 < len(capitulos) else len(texto)

        # Crear Span desde el texto original
        span = doc.char_span(start_char, end_char, alignment_mode="expand")

        if span:
            span._.numero = numero
            span._.titulo = titulo
            span._.longitud = len(span)
            spans.append(span)

    # Guardar la lista en el atributo del Doc
    doc._.capitulos = spans

    return doc
nlp.add_pipe("fragmentador_capitulos", last=True)


<function __main__.fragmentador_capitulos(doc)>

# Ejercicio: Segmentación en capítulos del tercer libro de Harry Potter

- Define la variable `nlp` con el modelo `es_core_news_md`.
- Añade el componente que has definirdo al pipeline de spaCy. Respecto a este pipeline, en el resto de esta notebook no será necesario utilizar el NER predefinido, ya que nuestro interés será detectar otro tipo de entidades que nosotros mismos vamos a definir. 
- Carga el tercer libro de Harry Potter a Partir de la línea 37. 
- Procesa el texto del tercer libro de Harry Potter con `nlp`, y comprueba que el `Doc` resultante contenga todos los capítulos segmentados y anotados correctamente.

In [29]:
if not spacy.tokens.Doc.has_extension("capitulos"):
    spacy.tokens.Doc.set_extension("capitulos", default=[])
nlp = spacy.load("es_core_news_md")
nlp.add_pipe("fragmentador_capitulos", last=True)
with open("libro3.txt", "r", encoding="utf-8") as archivo:
    lineas = archivo.readlines()[37:]  # Desde la línea 38
    texto = "".join(lineas)
doc = nlp(texto)


In [31]:
for capitulo in doc._.capitulos:
    print(f"Capítulo {capitulo._.numero}: {capitulo._.titulo} (Longitud: {capitulo._.longitud} caracteres)")

Capítulo 1: Lechuzas mensajeras (Longitud: 4418 caracteres)
Capítulo 2: El error de tía Marge (Longitud: 4651 caracteres)
Capítulo 3: El autobús noctámbulo (Longitud: 5433 caracteres)
Capítulo 4: El Caldero Chorreante (Longitud: 6465 caracteres)
Capítulo 5: El dementor (Longitud: 8178 caracteres)
Capítulo 6: Posos de té y garras de hipogrifo (Longitud: 8244 caracteres)
Capítulo 7: El boggart del armario ropero (Longitud: 5464 caracteres)
Capítulo 8: La huida de la señora gorda (Longitud: 6202 caracteres)
Capítulo 9: La derrota (Longitud: 6388 caracteres)
Capítulo 10: El mapa del merodeador (Longitud: 8671 caracteres)
Capítulo 11: La Saeta de Fuego (Longitud: 6762 caracteres)
Capítulo 12: El patronus (Longitud: 5809 caracteres)
Capítulo 13: Gryffindor contra Ravenclaw (Longitud: 5152 caracteres)
Capítulo 14: El rencor de Snape (Longitud: 6651 caracteres)
Capítulo 15: La final de quidditch (Longitud: 6964 caracteres)
Capítulo 16: La predicción de la profesora Trelawney (Longitud: 5275 ca

## Ejercicio: Detectar hechizos y añadirlos como entidades

Utilizando el objeto `PhraseMatcher` de spaCy, crea un sistema que detecte menciones de hechizos en un texto y los añada como nuevas entidades. Utiliza la siguiente lista de hechizos:

`hechizos = ["Expeliarmo", "Alohomora", "Expecto patronum", "Lumos"]`

Realiza este ejercicio de dos formas diferentes y razona qué está ocurriendo. 



In [38]:
import spacy
from spacy.matcher import PhraseMatcher

# Lista de hechizos
hechizos = ["Expeliarmo", "Alohomora", "Expecto patronum", "Lumos"]

# Cargar el modelo de SpaCy
nlp = spacy.load("es_core_news_md")

# Crear el PhraseMatcher y agregar las frases correspondientes
matcher = PhraseMatcher(nlp.vocab)
patterns = [nlp.make_doc(hechizo) for hechizo in hechizos]
matcher.add("HECHIZOS", None, *patterns)

# Función para añadir los hechizos como nuevas entidades
@Language.component("detectar_hechizos")
def detectar_hechizos(doc):
    doc.ents = []
    matches = matcher(doc)
    # Crear entidades en el doc para cada coincidencia
    for match_id, start, end in matches:
        span = Span(doc, start, end, label="HECHIZO")
        doc.ents = list(doc.ents) + [span]  # Añadir la nueva entidad al doc
    return doc

# Agregar el componente a la pipeline
nlp.add_pipe('detectar_hechizos', last=True)

<function __main__.detectar_hechizos(doc)>

1. Con el `texto` del tercer libro cargado tal cual

In [39]:
with open("libro3.txt", "r", encoding="utf-8") as archivo:
    lineas = archivo.readlines()[37:]  # Desde la línea 38
    texto = "".join(lineas)
doc = nlp(texto)

# Ver las entidades detectadas
for ent in doc.ents:
    if ent.label_ == "HECHIZO":  # Filtrar solo las entidades de tipo 'HECHIZO'
        print(f"Entidad: {ent.text}, Etiqueta: {ent.label_}")

Entidad: Lumos, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO


2. Haciendo la modificación `texto = texto.replace("", " ").replace("\x97", " ")` después de cargar el tercer libro.

In [40]:
with open("libro3.txt", "r", encoding="utf-8") as archivo:
    lineas = archivo.readlines()[37:]  # Desde la línea 38
    texto = "".join(lineas)
texto = texto.replace("", " ").replace("\x97", " ")
doc = nlp(texto)

# Ver las entidades detectadas
for ent in doc.ents:
    if ent.label_ == "HECHIZO":  # Filtrar solo las entidades de tipo 'HECHIZO'
        print(f"Entidad: {ent.text}, Etiqueta: {ent.label_}")

Entidad: Lumos, Etiqueta: HECHIZO
Entidad: Lumos, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Lumos, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Lumos, Etiqueta: HECHIZO
Entidad: Expeliarmo, Etiqueta: HECHIZO
Entidad: Expeliarmo, Etiqueta: HECHIZO
Entidad: Expeliarmo, Etiqueta: HECHIZO
Entidad: Expeliarmo, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etiqueta: HECHIZO
Entidad: Expecto patronum, Etique